In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./exampleNY.csv")
df

,Date,temperature_max,temperature_min,temperature_ave,rain,snow,extreme_events,sea_levels
0,192401,21.4,11.6,NaN,NaN,NaN,NaN,NaN
1,192402,15.8,5.9,NaN,NaN,NaN,NaN,NaN
2,192403,29.2,21.3,NaN,NaN,NaN,NaN,NaN
3,192404,40.2,30.0,NaN,NaN,NaN,NaN,NaN
4,192405,49.2,39.2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1215,202504,45.2,33.5,NaN,NaN,NaN,NaN,NaN
1216,202505,55.2,46.1,NaN,NaN,NaN,NaN,NaN
1217,202506,66.5,56.3,NaN,NaN,NaN,NaN,NaN
1218,202507,71.9,60.8,NaN,NaN,NaN,NaN,NaN


In [3]:
df["year"]=df["Date"].astype(str).str[:4]
df["month"]=df["Date"].astype(str).str[4:6]
df["state"]='NY'
df.drop('Date', axis=1, inplace=True) 

In [4]:
df=df[['state','year','month','temperature_ave','temperature_min','temperature_max','rain','snow','extreme_events','sea_levels']]

In [5]:
df

,state,year,month,temperature_ave,temperature_min,temperature_max,rain,snow,extreme_events,sea_levels
0,NY,1924,01,NaN,11.6,21.4,NaN,NaN,NaN,NaN
1,NY,1924,02,NaN,5.9,15.8,NaN,NaN,NaN,NaN
2,NY,1924,03,NaN,21.3,29.2,NaN,NaN,NaN,NaN
3,NY,1924,04,NaN,30.0,40.2,NaN,NaN,NaN,NaN
4,NY,1924,05,NaN,39.2,49.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1215,NY,2025,04,NaN,33.5,45.2,NaN,NaN,NaN,NaN
1216,NY,2025,05,NaN,46.1,55.2,NaN,NaN,NaN,NaN
1217,NY,2025,06,NaN,56.3,66.5,NaN,NaN,NaN,NaN
1218,NY,2025,07,NaN,60.8,71.9,NaN,NaN,NaN,NaN


In [6]:
df.to_json('NYdata.json',orient='records')